###STEP 1: PREPARING THE DATA

In [ ]:
import pandas as pd
import json

#Load the JSON file
file = "/content/lyrics_200.jl" #this file path needs to be set properly while running the notebook
data = []
with open(file, 'r') as f:
  for line in f:
    data.append(json.loads(line))

#Convert to pandas dataframe
df = pd.DataFrame(data)
df.head()

,song,lyrics
0,Erykah-badu-kiss-me-on-my-neck-lyrics,"\n\n[Production by J Dilla, Erykah Badu, and J..."
1,Daniel-caesar-we-find-love-lyrics,\n\n[Verse 1]\nYou don't love me anymore\nLet'...
2,Florence-the-machine-hunger-lyrics,"\n\n[Intro]\nOoh, ooh, ooh-ooh, ooh\nOoh, ooh,..."
3,Michael-jackson-will-you-be-there-lyrics,\n\n[Verse 1]\nHold me\nLike the River Jordan\...
4,Partynextdoor-not-nice-lyrics,\n\n[Intro: PARTYNEXTDOOR]\nOh-oh-oh-oh\nOh-oh...


###STEP 2: TOKENIZATION

Tokenization is performed using the NLTK Library.
1.   There are multiple alternatives available, some of which are spaCy (for relatively larger datasets), transformers (by Hugging Face) and Gensim.
2.   After enough research, decided to experiment with NLTK library initially as it gives more control over each pre-processing step allowing more scope for experimentation with each of the tokenization techniques.
3.    Performed steps like case-folding (converting to lower case to ensure consistency), tokenizing (splitting text into individual tokens), stop-word removal (to reduce noise coming from words that do not carry any useful information about the context), punctuation removal (as they do not add value in search context), and stemming (reducing words to their root form).
4.    All these steps are essential to implement our basic retrieval model, Vector Space Model. It requires stop-word removal and punctuation removal as we do not want noise in our system. Also, there is a need for case-folding and stemming in VSM or any other models like BM25, where metrics like term frequency, inverse document frequency are used. This is because, considering words like "loving", "Love" and "loved" as three separate independent words in the vocabulary would reduce the accuracy of the model.
5.    Removal of stop words during the preprocess stage is not suitable for Boolean retrieval approach because when we preprocess the query, the operators like "and" and "or" are removed from the query, and these operators are essential to process the user query correctly.



In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

We skip the removal of stop-words step in the preprocess method, as we will use this preprocess method in Boolean retrieval approach. We do not need this method for VSM as it has an inbuilt tokenizer.

In [ ]:
def preprocess(text):
  #case-folding
  text = text.lower()

  #tokenize
  tokens = word_tokenize(text)

  #remove stop words
  '''
  stop_words = set(stopwords.words('english'))
  tokens = [word for word in tokens if word not in stop_words]
  '''

  #remove punctuation
  tokens = [word for word in tokens if word not in string.punctuation]

  #stemming
  stemmer = PorterStemmer()
  tokens = [stemmer.stem(word) for word in tokens]
  return tokens

df['processed_lyrics'] = df['lyrics'].apply(preprocess)
df.head()

,song,lyrics,processed_lyrics
0,Erykah-badu-kiss-me-on-my-neck-lyrics,"\n\n[Production by J Dilla, Erykah Badu, and J...","[product, by, j, dilla, erykah, badu, and, jam..."
1,Daniel-caesar-we-find-love-lyrics,\n\n[Verse 1]\nYou don't love me anymore\nLet'...,"[vers, 1, you, do, n't, love, me, anymor, let,..."
2,Florence-the-machine-hunger-lyrics,"\n\n[Intro]\nOoh, ooh, ooh-ooh, ooh\nOoh, ooh,...","[intro, ooh, ooh, ooh-ooh, ooh, ooh, ooh, ooh-..."
3,Michael-jackson-will-you-be-there-lyrics,\n\n[Verse 1]\nHold me\nLike the River Jordan\...,"[vers, 1, hold, me, like, the, river, jordan, ..."
4,Partynextdoor-not-nice-lyrics,\n\n[Intro: PARTYNEXTDOOR]\nOh-oh-oh-oh\nOh-oh...,"[intro, partynextdoor, oh-oh-oh-oh, oh-oh-oh-o..."


###STEP 3: BUILD INVERTED INDEX

The inverted index is created using a defaultdict with lists as default values.
1.   The code iterates through each document in the dataset, using the enumerate function to keep track of the document index.
2.   For each token in a document, the document's index is added to the list asssociated with that token in the inverted index.
3.    It allows for fast lookup as the time complexity for retrieving documents containing a specific token is O(1).
3.    We will use it in Boolean retrival to process every query token in an efficient manner.
4.    It can also be used in VSM to efficiently calculate document frequency (a metric used to calculate inverse document frequency, i.e., IDF). However, we will keep our implementation simple and use the inbuit tf-idf vectorizer to obtain the tf-idf matrix.



In [ ]:
from collections import defaultdict

#inverted_index maps a given token to a list of indices of documents that this token appears in
inverted_index = defaultdict(list)
for idx, tokens in enumerate(df['processed_lyrics']):
  for token in tokens:
    #checks for repetitions
    if idx not in inverted_index[token]:
      inverted_index[token].append(idx)

# print(inverted_index) #prints the mapping of each token to a list of all documents that contain this token!

###RETRIEVAL MODELS AND RESULTS

VECTOR SPACE MODEL

Below is the implementation for Vector Space Model (VSM).


1.   It has an inbuilt tokenizer, hence we directly pass lyrics to the vectorizer.
2.   tf-idf matrix is built based on the lyrics dataset, query is converted into a vector based on the vocabulary built from the dataset.
3.   we perform cosine similarity (dot product) between the query vector and the tf-idf matrix to get a vector of shape 1*200, each value in this vector corresponds to the relevance of each song (document) to the query.
4.   We display the top 10 ranked matches!





In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#vectorizing the text
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['lyrics'])

In [ ]:
#user query
query = "dreams and ambitions"
query_vector = vectorizer.transform([query])

In [ ]:
cosine_sim = cosine_similarity(query_vector, tfidf_matrix)
# print(cosine_sim)
print(cosine_sim.shape)
ranking = cosine_sim.argsort()[0][::-1]

# print(ranking)

(1, 200)


In [ ]:
for rank, idx in enumerate(ranking[:10]):
  print(f"Rank {rank+1}: {df.iloc[idx]['song']}")
  print(f"Lyrics: {df.iloc[idx]['lyrics'][:100]}...\n\n")

Rank 1: Skylar-grey-tower-dont-look-down-lyrics
Lyrics: 

[Chorus]
You're high up on the tower
Now don’t look down
I will be ok here on the ground
And you c...


Rank 2: Alicia-keys-empire-state-of-mind-part-ii-broken-down-lyrics
Lyrics: 

[Intro]
Ooooh, New York!
Ooooh, New York!

[Verse 1]
Grew up in a town that is famous as a place o...


Rank 3: Green-day-macys-day-parade-lyrics
Lyrics: 

[Verse 1]
Today's the Macy's Day Parade
The night of the living dead is on its way
With a credit r...


Rank 4: Earl-sweatshirt-playing-possum-lyrics
Lyrics: 

[Cheryl Harris + Keorapetse Kgositsile]
To my mentors and comrades in arms
Those presence and thos...


Rank 5: Rihanna-a-girl-like-me-lyrics
Lyrics: 

[Intro]
Ohh
Ohh

[Verse 1]
Some girls play the game
They all walk and talk and they dress the same...


Rank 6: Muse-liquid-state-lyrics
Lyrics: 

[Verse 1]
Take me for a ride
Break me up and steal what’s left inside
And hope and pray iniquity h...


Rank 7: Florence-the-machine-all-this-and

BOOLEAN RETRIEVAL

Boolean retrieval is not a ranked model, hence we only display all the documents which are relevant to the given query.
1. In this simple implementation, queries can be of the following forms:
  *   W1 and W2 and W3 and ..
  *   W1 or W2 or W3 or ... (where W1, W2, W3, .. are words)
2. We use inverted indexing to efficiently fetch documents containing specific tokens
3. This method isn't that useful as in most cases it is only searching for the exact words in documents, for example, if we have a query with more than 3-4 words with "and" operators, there are chances that there are no documents having all these words.
4. Hence, VSM is better than this model, as we get ranked matches, even if the query does not match with any of the documents very well.
5. This approach is more useful in scenarios where the users know the exact terms they want to search for.



In [ ]:
def boolean_retrieval(query):
  #Assume query is either of the form: word1 and word2 and word3 ....
  #or of the form: word1 or word2 or word3 ....
  tokens = preprocess(query)
  print(tokens)
  result_set = set() #a set is being used to ensure no duplicates
  current_op = None #to keep track of the current operator

  for token in tokens:
    if token == "and":
      current_op = "AND"
    elif token == "or":
      current_op = "OR"
    else:
      documents = set(inverted_index.get(token,[])) #this is a set of all documents containing the token
      if current_op == None: #handling the first token in a query
        result_set = documents
      elif current_op == "AND":
        result_set = result_set.intersection(documents) #we take intersection of all sets of documents corresponding to every token in the query
      elif current_op == "OR":
        result_set = result_set.union(documents) #we take union here, as it is an "or" operator

      current_op = None
  return result_set

def print_result(result_set):
  for idx in result_set:
    print(f"Song: {df.iloc[idx]['song']}")
    print(f"Lyrics: {df.iloc[idx]['lyrics'][:100]}...\n")

#user query
query = "dreams and ambitions"
print_result(boolean_retrieval(query))

['dream', 'and', 'ambit']
Song: Skylar-grey-tower-dont-look-down-lyrics
Lyrics: 

[Chorus]
You're high up on the tower
Now don’t look down
I will be ok here on the ground
And you c...



BM25 (Best Matching 25)

We first initialize BM25 with the list of tokenized documents. BM25 has an inbuilt function called get_scores which directly results in similarity scores of a query with each of the documents.

When manually checked, this method almost performed similar to VSM (or even better as it is a probabilistic ranking model)

NOTE: we use preprocess function for the dataset as well as the query in this case. Based on current implementation of this function, we are not eliminating the stop words, but if we eliminate stop words, the performance will be even better as we will  be focussing only on the words that carry useful contextual information

In [ ]:
pip install rank-bm25

In [ ]:
from rank_bm25 import BM25Okapi

data = df['processed_lyrics'].tolist()
bm25 = BM25Okapi(data) #we initialize BM25 with list of tokenized documents

query = "dreams and ambitions"
query_tokens = preprocess(query)

bm25_scores = bm25.get_scores(query_tokens)
enumerated_scores = list(enumerate(bm25_scores)) #enumerated scores consists of pairs of each of the scores and their corresponding documents.
# print(enumerated_scores)
sorted_scores = sorted(enumerated_scores, key=lambda x: x[1], reverse=True)
print(sorted_scores)

[(81, 12.319695174777632), (110, 5.9484220462049), (182, 5.479722673381405), (69, 5.461145516237483), (187, 5.414601024277422), (176, 5.359129101174232), (132, 5.244790443166585), (27, 5.078519891374713), (56, 4.859649648562884), (32, 4.841369453808925), (169, 4.773298525303748), (22, 4.620101447863762), (143, 4.4265727425626284), (171, 4.413102954042591), (95, 4.4094919022490275), (51, 4.022441634528485), (94, 4.004485248143685), (125, 3.8678470663070663), (15, 3.815442726740629), (10, 3.657042173328526), (19, 3.5286608196374116), (5, 3.27385282618958), (67, 3.1948946552709927), (6, 3.1786630186155325), (155, 2.5881393588847947), (184, 2.5365814671378852), (158, 2.520707152733329), (178, 2.51029245083039), (115, 2.503374007066065), (29, 2.482007142530334), (129, 2.4773011301206376), (37, 2.475577952086376), (24, 2.475556872106679), (16, 2.465815335759918), (42, 2.4629099528496874), (40, 2.462881334251264), (57, 2.4562702340464626), (71, 2.454840753567499), (84, 2.4534409164151976), (3

In [ ]:
def print_bm25_results(ranking,top_k):
  for rank, (idx, score) in enumerate(ranking[:top_k]):
    print(f"Rank {rank+1}: {df.iloc[idx]['song']}")
    # print(f"Score: {score}")
    print(f"Lyrics: {df.iloc[idx]['lyrics'][:100]}...\n")

print_bm25_results(sorted_scores,10)

Rank 1: Skylar-grey-tower-dont-look-down-lyrics
Lyrics: 

[Chorus]
You're high up on the tower
Now don’t look down
I will be ok here on the ground
And you c...

Rank 2: David-bowie-boss-of-me-lyrics
Lyrics: 

[Verse 1]
Tell me when you're sad
I wanna make it cool again
I know you're feeling bad
Tell me whe...

Rank 3: Imagine-dragons-thunder-lyrics
Lyrics: 

[Verse 1]
Just a young gun with a quick fuse
I was uptight, wanna let loose
I was dreaming of bigg...

Rank 4: Yonas-lights-remix-lyrics
Lyrics: 

[Hook: Ellie Goulding]
And I'm not sleeping now, the dark is too hard to beat
And I'm not keeping ...

Rank 5: 2-chainz-forgiven-lyrics
Lyrics: 

[Intro: Marsha Ambrosius]
And we'll introduce you to the starting lineup
We're gonna introduce now...

Rank 6: Alessia-cara-the-other-side-lyrics
Lyrics: 

[Verse 1]
In my room, staying up late
And I'm planning my escape
Cause I know I can't stay
In my v...

Rank 7: Green-day-macys-day-parade-lyrics
Lyrics: 

[Verse 1]
Today's the Macy's Day Pa

COLBERT

This approach performs the best when compared with the above approaches as it considers embeddings instead of vectors. Word embeddings try to capture the contextual semantics of sentences and hence it is able to perform better semantic search.



In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd

#initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("colbert-ir/colbertv2.0")
model = AutoModel.from_pretrained("colbert-ir/colbertv2.0")

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
documents = list(df['lyrics'])
doc_embeddings = []
for doc in documents:
  inputs = tokenizer(doc, return_tensors="pt", truncation=True, padding=True) #tokenizing each document before passing it to the model
  outputs = model(**inputs) #passing inputs in a form that can be processed by the model
  doc_embedding = outputs.last_hidden_state[:, 0, :].detach()
  doc_embeddings.append(doc_embedding)
doc_embeddings = torch.cat(doc_embeddings, dim=0)

#encode the query
query = "dreams and ambitions"
query_inputs = tokenizer(query, return_tensors="pt", truncation=True, padding=True)
query_outputs = model(**query_inputs)
query_embedding = query_outputs.last_hidden_state[:, 0, :].detach()

#compute cosine similarity
cosine_scores = torch.nn.functional.cosine_similarity(query_embedding, doc_embeddings)

In [ ]:
#get top-k results
top_k = torch.topk(cosine_scores, k=10)

print(top_k)

#display results
def display_colbert_results(top_k):
  for rank, idx in enumerate(top_k.indices):
    idx = int(idx)
    print(f"Rank {rank+1}: {df.iloc[idx]['song']}")
    print(f"Lyrics: {df.iloc[idx]['lyrics'][:100]}") #printing the first 100 characters of every song's lyrics

display_colbert_results(top_k)


torch.return_types.topk(
values=tensor([0.6359, 0.6108, 0.5886, 0.5797, 0.5592, 0.5528, 0.5514, 0.5494, 0.5459,
        0.5447]),
indices=tensor([122,  36, 177, 155,  93, 162, 188, 143,  15, 132]))
Rank 1: Lana-del-rey-burnt-norton-interlude-lyrics
Lyrics: 

[Interlude]
Time present and time past
Are both perhaps present in time future
And time future con
Rank 2: Marina-the-archetypes-lyrics
Lyrics: 

Housewife, Beauty Queen, Homewrecker, Idle Teen
The ugly years of being a fool, ain't youth meant 
Rank 3: Muse-mk-ultra-lyrics
Lyrics: 

[Verse 1]
The wavelength gently grows
Coercive notions re-evolve
A universe is trapped inside a te
Rank 4: Sza-pray-lyrics
Lyrics: 

[Produced by Felix Snow]

Pray for yourself one time
One time, time

Young Medusa
Never sure, i ju
Rank 5: Aesop-rock-rings-lyrics
Lyrics: 

[Verse 1]
Used to draw
Hard to admit that I used to draw
Portraiture and the human form
Doodle of 
Rank 6: Coldplay-rastafarian-targaryen-lyrics
Lyrics: 

Game of what now?

I'm a ras

NOTE: When manually compared the results between the three ranked appraoches namely, VSM, BM25 and ColBERT:
  *   Kept the query fixed as "dreams and ambitions" for all approaches
  *   Displayed the top 10 best ranked songs for every approach given a query.
  *   Tested manually if each of the songs is relevant or not relevant to the query.
  *   The number of relevant documents was around 8 out of 10 in the case of ColBERT whereas it was around 6-7 out of 10 in the cases of VSM and BM25. Also intuitively, since it is important that we capture the contextual understanding of the whole lyrics of a given song, it is ideal to choose word embeddings over vectors. In case of BM25 and VSM, we consider each of the tokens as independent entities, not taking into consideration the dependencies between them.

###FUTURE IMPROVEMENTS

Some future improvements are listed below:
*   The boolean retrieval model is not a ranked model. But it can be combined with the vector space model by first finding the tf-idf values for each query token in a given query, and then finding the relevance of the whole query with the set of documents we get as output from the boolean retrieval model. By doing this, we can inculcate some ranking amongst the set of documents we get as output.
*   In boolean retrieval, query processing can be expanded by supporting complex boolean expressions like "dreams AND (ambitions OR failures)"
*   So far, the ColBERT model performs the best. We can further improve it's performance by fine-tuning the model on a huge labelled dataset of song lyrics (each song is labelled with a relevant query)!
